In [ ]:
#IMPORTING KAGGLE DATASETS

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.layers import Dropout

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
df_train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
df_test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
df_submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [ ]:
def jaccard(str1, str2): 
    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
df_train.head()

In [ ]:
df_train['Num_words_text'] = df_train['text'].apply(lambda x:len(str(x).split())) 

In [ ]:
df_train.head()

In [ ]:
X = df_train.dropna()

# Get training data
X = df_train.drop('sentiment', axis = 1)

#  Get target label
y = df_train['sentiment']

In [ ]:
# copying data for further processing
messages = X.copy()

# The reset_index() function is used to generate a new DataFrame or Series with the index reset
messages.reset_index(inplace = True)

#downloading stop words
nltk.download('stopwords')

In [ ]:
ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    # replace with space words other than a-1, A-Z
    
    review = re.sub('[^a-zA-Z]', ' ', str(messages['text'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
# vocabulray size
voc_size = 5000
# One Hot Encoding
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [ ]:
sent_length = 30
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

In [ ]:
num_labels = len(set(df_train['sentiment']))

In [ ]:
embedding_vector_features = 40

## Creating model
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(num_labels,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

In [ ]:
from sklearn import preprocessing

# encode label to int
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

X_final = np.array(embedded_docs)
y_final = np.array(y)

from keras.utils import to_categorical
y_final = to_categorical(y_final)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
his = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 50, batch_size = 64)

In [ ]:
def return_x_y(X):
    
    # Drop Nan Values
    X = X.fillna(0)
    
    messages = X.copy()

    messages.reset_index(inplace = True)

    # Dataset Preprocessing
    ps = PorterStemmer()

    corpus = []

    for i in range(0, len(messages)):
        # replace with space words other than a-1, A-Z

        review = re.sub('[^a-zA-Z]', ' ', str(messages['text'][i]))
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)

    # vocabulray size
    voc_size = 5000

    onehot_repr = [one_hot(words, voc_size) for words in corpus]

    # Embedding Representation
    # making all sentences of same length
    sent_length = 30
    embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

    X_final = np.array(embedded_docs)
    
    
    return X_final, X


In [ ]:
# test data and pre-processing
X_test,X_test_drop = return_x_y(df_test)

In [ ]:
# making prediction
y_pred_test = model.predict_classes(X_test)

In [ ]:
len(X_test_drop['textID']), len(y_pred_test)

In [ ]:
df_sub = pd.DataFrame()
df_sub['textID'] = X_test_drop['textID']
df_sub['selected_text'] = X_test_drop['text']
df_sub['sentiment_predicted'] = le.inverse_transform(y_pred_test)
df_sub['sentiment_actual'] = X_test_drop['sentiment']

In [ ]:
df_sub.head(50)

In [ ]:
df_sub.drop(['sentiment_predicted','sentiment_actual'],inplace=True,axis=1)

In [ ]:
# Visualizing the data
df_sub.head()

In [ ]:
df_submission['selected_text'] = df_sub['selected_text']
df_submission.to_csv("submission.csv", index=False)
display(df_submission.head(10))